# 1. Introdução

Neste notebook usamos uma rede U-Net (ou semelhante) pré-treinada para aplicar segmentação semântica em uma imagem real.

Vamos utilizar um modelo da `torchvision` (DeepLabV3+ com ResNet50) como exemplo prático.

In [ ]:
# Instalar pacotes (se necessário no Colab)
# !pip install torchvision matplotlib
import torch
import torchvision.transforms as T
from torchvision.models.segmentation import deeplabv3_resnet50
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from google.colab.patches import cv2_imshow
import cv2


In [ ]:
# Verifica se já foram baixadas as imagens do drive, baixando-as e descompactando se necessário
! [ ! -d "/content/cachorro.jpeg" ] && gdown -O /content/cachorro.jpeg "1xQdpWJ4fU7RUMltryqeNqaF31gKhYYzZ"

! [ ! -d "/content/carro.jpg" ] && gdown -O /content/carro.jpg "16F9AkZybPG3ppcoOboYlfvRmTK0g77wV"

! [ ! -d "/content/pessoas.jpg" ] && gdown -O /content/pessoas.jpg "19Q9M_vCwIxyUK4C45F2ZjRZcKUoBUugq"


In [ ]:
# Carregar modelo pré-treinado (21 classes da PASCAL VOC)
model = deeplabv3_resnet50(pretrained=True)
model.eval();

In [ ]:
# Mapear classes para cores
def decode_segmap(image, nc=21):
    # Mapa de cores: cada índice de classe (0 a 20) é associado a uma cor RGB.
    # Isso permite que cada classe seja representada visualmente por uma cor distinta.
    label_colors = np.array([
        (0, 0, 0),
        (128, 0, 0),
        (0, 128, 0),
        (128, 128, 0),
        (0, 0, 128),
        (128, 0, 128),
        (0, 128, 128),
        (128, 128, 128),
        (64, 0, 0),
        (192, 0, 0),
        (64, 128, 0),
        (192, 128, 0),
        (64, 0, 128),
        (192, 0, 128),
        (64, 128, 128),
        (192, 128, 128),
        (0, 64, 0),
        (128, 64, 0),
        (0, 192, 0),
        (128, 192, 0),
        (0, 64, 128)
    ])

    # Cria 3 matrizes vazias do mesmo tamanho da imagem de classes (H×W), para os canais RGB
    r = np.zeros_like(image).astype(np.uint8)  # canal vermelho
    g = np.zeros_like(image).astype(np.uint8)  # canal verde
    b = np.zeros_like(image).astype(np.uint8)  # canal azul

    # Para cada classe l (de 0 até nc-1):
    for l in range(0, nc):
        # Cria uma máscara booleana onde a imagem de classes == l
        idx = image == l
        if idx.any():
          print(l)

        # Preenche os canais RGB nas posições onde a classe é l, com a cor correspondente
        r[idx] = label_colors[l, 0]
        g[idx] = label_colors[l, 1]
        b[idx] = label_colors[l, 2]

    # Empilha os canais r, g, b em um único array 3D com shape (H, W, 3)
    rgb = np.stack([r, g, b], axis=2)

    return rgb

In [ ]:
# Carregar imagem (ex: pessoa, animal, vegetação)
img_path = 'cachorro.jpeg'
# img_path = 'pessoas.jpg'
# img_path = 'carro.jpg'

img = Image.open(img_path).convert('RGB')

# Pré-processamento
preprocess = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
input_tensor = preprocess(img).unsqueeze(0)  # shape: (1, 3, H, W)

display(img)

# Fazer inferência
with torch.no_grad():
    output = model(input_tensor)['out'][0]  # shape: (21, H, W)
pred = torch.argmax(output, dim=0).numpy()

segmap = decode_segmap(pred)
cv2_imshow(cv2.cvtColor(segmap, cv2.COLOR_RGB2BGR))